<a href="https://colab.research.google.com/github/sqbitegh/Colabs/blob/main/ColabQwenServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#script works on T4 only
!pip install flask-ngrok
!pip install flask
#apparently not needed

In [ ]:
pip install git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830 accelerate

In [ ]:
#!pip install pyngrok==4.1.1
!pip install pyngrok
pip install qwen-vl-utils[decord]
pip install autoawq

In [ ]:
import requests
from io import BytesIO
from PIL import Image
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct-AWQ", torch_dtype="auto", device_map="auto"
)

# We recommend enabling flash_attention_2 for better acceleration and memory saving, especially in multi-image and video scenarios.
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-7B-Instruct",
#     torch_dtype=torch.bfloat16,
#     attn_implementation="flash_attention_2",
#     device_map="auto",
# )

# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct-AWQ")




In [ ]:
def process_message_single(prompt, question, path_to_video):
  messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "file:///content/sample_data/5fps_10s_quarter_video_example.mp4",
                "max_pixels": 360 * 420,
                "fps": 1.0,
            },
            {"type": "text", "text": "Is there person visible. Answer with 1 word: yes or no."},
        ],
    }
  ]
  # Preparation for inference
  text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
  )
  image_inputs, video_inputs = process_vision_info(messages)
  inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
  )
  inputs = inputs.to("cuda")

  # Inference
  generated_ids = model.generate(**inputs, max_new_tokens=128)
  generated_ids_trimmed = [
      out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
  ]
  output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
  )
  print(output_text)
  return output_text



In [ ]:


from pyngrok import ngrok
from flask import Flask, request, Response

# token got to https://dashboard.ngrok.com/get-started/your-authtoken
ngrok_token = "yyyyy"

port = 5000
ngrok.set_auth_token(ngrok_token)

app = Flask(__name__)

@app.route("/")
def root():
  return "Hello world colab pyngrok!"

@app.route("/req1", methods=['POST'])
def req1():
    if 'video' not in request.files or 'text' not in request.form:
        return "Missing video or text data", 400

    video_file = request.files['video']
    text_data = request.form['text']

    # Print the received data
    print(f"Received text data: {text_data}")
    print(f"Received video file: {video_file.filename}")

    # You can process the video file here, e.g., save it to disk
    videoPath = "sample_data/"+video_file.filename
    video_file.save(videoPath)
    response = process_message_single("", text_data, videoPath):
    # Display the video in the notebook (not possible with ngrok)
    # display(HTML(f'<video controls><source src="data:video/mp4;base64,{base64.b64encode(video_file.read()).decode()}" type="video/mp4"></video>'))


    # Send the text data back as response
    return "Response to: " + text_data + " is : " + response




if __name__ == "__main__":
  try:
    app.logger.info('Hello from Flask logger!')
    public_url = ngrok.connect(port).public_url
    print(public_url)
    app.logger.info(public_url)
    app.run(port=port)
  finally:
    ngrok.disconnect(public_url=public_url)



https://0bbf-35-227-96-102.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


PyngrokNgrokURLError: ngrok client exception, URLError: [Errno 111] Connection refused